In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import sys
import os
import scipy.sparse as sp
import  pickle as pkl
import networkx as nx

# DataLoad


In [2]:
dataset_name = "cora"
def preprocess_features(features):
    """
    行归一化 每行除以每行的和
    """
    row_sum = np.array(features.sum(1))
    reverse_row_sum = np.power(row_sum, -1).flatten()
    reverse_row_sum[np.isinf(reverse_row_sum)] = 0.
    new_features = sp.diags(reverse_row_sum).dot(features)
    return new_features

def load_data(dataset_name):
    """
    dataset_name = cora, citeseet, pubmed
    # 都是pickle文件
    .x: 训练集的feature vectors, scipy.sparse.csr.csr_matrix
    .tx: 测试集feature vectors
    .allx: labeled 和unlabeled training feature vectors
    .y: one-hot labels of .x
    .ty: one-hot labels of .tx
    .ally: one-hot labels of .allx
    .graph: {index: [index_of_neighbor_nodes]}
    .test.index: 测试实例的id
    """
    names = ['x', 'tx', 'allx','y','ty','ally', 'graph']
    objects = {}
    for name in names:
        with open("data/ind.{}.{}".format(dataset_name, name), 'rb') as f:
            objects[name] = pkl.load(f, encoding='latin1')

    # 测试节点的id
    with open("data/ind.{}.test.index".format(dataset_name), 'r') as f:
        index = []
        for line in f.readlines():
            index.append(int(line.strip()))
        
    test_index_reorder = np.sort(index)  # 测试集的ids
    print(objects['x'].shape)   # (140,1433)    训练集中带有标签的部分
    print(objects['tx'].shape)  # (1000, 1433)
    print(objects['allx'].shape)  # (1708, 1433)  训练集
    print(len(test_index_reorder))  # 1000
    
    
    whole_features = sp.vstack((objects['allx'], objects['tx'])).tolil()  # tx 的顺序是乱序的 index 转为lil稀疏矩阵 可以直接运算
    


    whole_features[index,:] = whole_features[test_index_reorder,:]

    adj = nx.adjacency_matrix(nx.from_dict_of_lists(objects['graph']))  # return a scipy sparse matrix

    whole_labels = np.r_[objects['ally'], objects['ty']]
    whole_labels[index,:] = whole_labels[test_index_reorder,:]  

    whole_features = preprocess_features(whole_features)
    # 训练集featrues

    X_train = whole_features[range(objects['x'].shape[0])]
    y_train = whole_labels[range(objects['y'].shape[0])]

    X_valid = whole_features[objects['x'].shape[0]: objects['x'].shape[0] + 500]
    y_valid = whole_labels[objects['y'].shape[0]: objects['y'].shape[0] + 500]

    X_test = whole_features[test_index_reorder]
    y_test = whole_labels[test_index_reorder]

    return adj, whole_features ,X_train, X_valid, X_test, y_train, y_valid, y_test



In [3]:
adj, whole_features, X_train, X_valid, X_test, y_train, y_valid, y_test = load_data(dataset_name)

(140, 1433)
(1000, 1433)
(1708, 1433)
1000


In [4]:
print(adj.shape)
print(type(adj))

(2708, 2708)
<class 'scipy.sparse.csr.csr_matrix'>


In [5]:
print(whole_features.shape)
print(type(whole_features))

(2708, 1433)
<class 'scipy.sparse.csr.csr_matrix'>


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(140, 1433)
(140, 7)
(500, 1433)
(500, 7)
(1000, 1433)
(1000, 7)


## 计算拉普拉斯矩阵

In [7]:
num_node = adj.shape[0]

def graph_laplacian(adj):
    """
    adj: sparse.csr_matrix
    """
    adj = adj + sp.eye(num_node)
    D_ii = np.power(np.array(adj.sum(axis = 1)).flatten(), -0.5)
    D_hat = sp.diags(D_ii)
    lapacian = D_hat @ adj @ D_hat
    return lapacian


In [8]:
graph_laplacian = graph_laplacian(adj)
type(graph_laplacian)

scipy.sparse.csr.csr_matrix

# Build GCN Model

## 定义GCN的层次

其中: dropout rate for all layers, L2 regularization factor for the first GCN layer and number of hiddenunits

In [9]:
class GraphCNN(keras.layers.Layer):
    def __init__(self, input_dim, 
                       output_dim, 
                       sparse_inputs = False,
                       dropout = 0.,
                       bias = False,
                       activation = keras.activations.relu, 
                       **kwargs):
        super(GraphCNN,self).__init__(**kwargs) 
        self.activation = activation
        self.dropout = dropout
        self.sparse_inputs = sparse_inputs
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.biases = bias

    def build(self, input_shape):
        """
        定义层内参数
        input_shape (features, graphLaplacian)
        """
        self.Weight_per_layer = self.add_weight(name = "weight",
                                                shape = (input_shape[0][1], self.output_dim),
                                                initializer = keras.initializers.glorot_uniform,
                                                trainable = True)
        if self.biases:
            self.bias_per_layer = self.add_weight(name = "bias",
                                                  shape = (input_shape[0][1],),
                                                  initializer = keras.initializers.zeros,
                                                  trainable = True)
        super(GraphCNN,self).build(input_shape)


    def _sparse_csr_matrix_to_TFSparseTensor(self, X):
        if sp.isspmatrix_csr(X):
            X = X.tocoo()
        row = X.row
        col = X.col
        pos = np.c_[row,col]
        data = X.data
        shape = X.shape
        # SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
        return tf.SparseTensor(pos, data, shape)

    def call(self, inputs):
        """
        x: (features, GraphLaplacian)
        dropout before input to next layer
        features is a SparseTensor
        GraphLaplacian is a csr_matrix
        """
        features, GraphLaplacian = inputs
        
        # if training is not False and self.sparse_inputs:
        #     # drop out
        #     features = None
        # elif training is not False:
        #     features = tf.nn.dropout(x, self.dropout)
        # print(type(features))
        # print(type(GraphLaplacian))
        L_dot_X = list()
        if self.sparse_inputs:
            for i in GraphLaplacian:
                L_dot_X.append(tf.sparse.sparse_dense_matmul(np.array(i.todense()).astype('float32'), features))
        else:
            for i in GraphLaplacian:
                L_dot_X.append(tf.matmul(np.array(i.todense()).astype('float32'), features)) 

        return self.activation(L_dot_X @ self.Weight_per_layer)

        # # 加快运行
        # L_dot_X = []
        # for i in GraphLaplacian:
            
        #     L_dot_X.append(tf.sparse.sparse_dense_matmul(i.todense(), features))
        
        # return self.activation(tf.matmul(np.array(L_dot_X), self.Weight_per_layer))



class Sparse_Dropout(keras.layers.Layer):
    def __init__(self, rate, noise_shape, **kwargs):
        """
        一部分feature 置为0
        rate: dropout rate
        noise: num of nonzero features
        """
        super(Sparse_Dropout,self).__init__()
        self.rate = rate
        self.noise_shape = noise_shape  

    def build(self, input_shape):
         super(Sparse_Dropout,self).build(input_shape)

    def call(self, inputs, training = True):
        # input is a tf.Sparse Tensor
        if training:
            retain_prob = tf.random.uniform(shape=[self.noise_shape[0]])   # 每个feature被保留的概率
            retain = (retain_prob >= self.rate)
            pre_out = tf.sparse.retain(inputs, retain)
            return pre_out * (1./ (1-self.rate))
        else: 
            return inputs


## 定义模型

In [10]:
class GCN(keras.models.Model):
    def __init__(self, input_dim, output_dim,  hidden_dim, num_nonzero_feature, sparse_input = True, dropout = 0.,**kwargs):
        super(GCN, self).__init__(**kwargs)
        print('input dim:', input_dim)
        print('output dim:', output_dim)
        print('num_features_nonzero:', num_nonzero_feature)

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.sparse_input = sparse_input
        self.dropout = dropout
        self.dropout1 = Sparse_Dropout(rate = self.dropout, noise_shape = num_nonzero_feature)
        self.GraphCNN1 = GraphCNN(input_dim = self.input_dim, 
                                 output_dim = self.hidden_dim, 
                                 sparse_inputs = self.sparse_input, 
                                 dropout = self.dropout,
                                 activation = keras.activations.relu)
        self.dropout2 = keras.layers.Dropout(self.dropout)
        self.GraphCNN2 = GraphCNN(input_dim = self.hidden_dim, 
                                 output_dim = self.output_dim, 
                                 dropout = self.dropout,
                                 activation = keras.activations.softmax)

        # self.activation = keras.layers.Activation("softmax")

    def call(self, inputs, training = True):
        features, graph_laplacian = inputs
        dropout1 = self.dropout1(features,training = training) # output(2708, 1433)
        gcn1 = self.GraphCNN1((dropout1,graph_laplacian))
        dropout2 = tf.squeeze(self.dropout2(gcn1))                    # (2708, 1, 16)
        gcn2 = self.GraphCNN2((dropout2,graph_laplacian))
        # out = self.activation(gcn2)
        return tf.squeeze(gcn2)


In [11]:
# transform sp.sparse.csr_matrix to tf.SparseTensor
def sparse_csr_matrix_to_TFSparseTensor(X):
    if sp.isspmatrix_csr(X):
        X = X.tocoo()
    row = X.row
    col = X.col
    pos = np.c_[row,col]
    data = X.data
    shape = X.shape
    # SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
    return tf.SparseTensor(pos, data, shape)

features = sparse_csr_matrix_to_TFSparseTensor(whole_features) 
num_nonzero_value = features.values.shape.as_list()

In [12]:
num_nonzero_value[0]

49216

In [13]:
features.shape[0]

2708

In [19]:
gcn_model = GCN(input_dim = features.shape[1], 
                output_dim = y_train.shape[1], 
                hidden_dim = 16, 
                num_nonzero_feature = num_nonzero_value,
                dropout = 0.5)

input dim: 1433
output dim: 7
num_features_nonzero: [49216]


## 定义loss

In [20]:
epochs = 60
optimizer = keras.optimizers.Adam(lr = 1e-2)

def compute_accuracy(y_true, y_pred):
    correct_prediction = tf.equal(tf.argmax(y_true, 1), tf.argmax(y_pred, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    return tf.reduce_mean(accuracy_all)

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y_train, gcn_model((features, graph_laplacian), training = True)[:y_train.shape[0]]))
        train_acc = compute_accuracy(y_train, gcn_model((features, graph_laplacian))[:y_train.shape[0]])
    grads = tape.gradient(loss, gcn_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, gcn_model.trainable_variables))

    val_output = gcn_model((features, graph_laplacian), training = False)[y_train.shape[0]: y_train.shape[0] + 500]
    val_loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y_valid, val_output))
    val_acc = compute_accuracy(y_valid, val_output)

    print("epoch: ", epoch, " train loss: ", float(loss.numpy()), " train acc: ", float(train_acc.numpy()), " val loss: ", float(val_loss.numpy()), " val acc: ", float(val_acc.numpy()))



epoch:  0  train loss:  1.9462742805480957  train acc:  0.10000000149011612  val loss:  1.9434977769851685  val acc:  0.3319999873638153
epoch:  1  train loss:  1.9410849809646606  train acc:  0.37142857909202576  val loss:  1.9400094747543335  val acc:  0.4440000057220459
epoch:  2  train loss:  1.9356205463409424  train acc:  0.5214285850524902  val loss:  1.9356262683868408  val acc:  0.5099999904632568
epoch:  3  train loss:  1.9281036853790283  train acc:  0.6285714507102966  val loss:  1.9306448698043823  val acc:  0.5619999766349792
epoch:  4  train loss:  1.9195727109909058  train acc:  0.6571428775787354  val loss:  1.9253590106964111  val acc:  0.5879999995231628
epoch:  5  train loss:  1.910069227218628  train acc:  0.699999988079071  val loss:  1.919954538345337  val acc:  0.5899999737739563
epoch:  6  train loss:  1.9002541303634644  train acc:  0.7142857313156128  val loss:  1.9143290519714355  val acc:  0.593999981880188
epoch:  7  train loss:  1.8874832391738892  train 

## Test

In [22]:
test_output = gcn_model((features, graph_laplacian), training = False)[-y_test.shape[0]:]
test_loss = tf.reduce_mean(keras.losses.categorical_crossentropy(y_test, test_output))
test_acc = compute_accuracy(y_test, test_output)

print("test loss: ", float(test_loss.numpy()), " test_acc: ", float(test_acc.numpy()))

test loss:  1.177369475364685  test_acc:  0.765999972820282
